In [168]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression, LinearRegression, Lasso, Ridge
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score, f1_score
from sklearn.svm import SVC, SVR
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA

In [132]:
df = pd.read_excel('Real estate valuation data set.xlsx')
print(df.columns)

Index(['No', 'X1 transaction date', 'X2 house age',
       'X3 distance to the nearest MRT station',
       'X4 number of convenience stores', 'X5 latitude', 'X6 longitude',
       'Y house price of unit area'],
      dtype='object')


In [173]:
y = df['Y house price of unit area']
X = df.drop(columns=['Y house price of unit area', 'No'])
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
lr = RandomForestRegressor()
lr.fit(x_train, y_train)
y_pred = lr.predict(x_test)
print(mean_squared_error(y_test, y_pred))
print(r2_score(y_test, y_pred))

32.45057828122155
0.8065651663095817


In [174]:
y = df['Y house price of unit area']
X = df.drop(columns=['Y house price of unit area', 'No'])
num_cols = X.columns
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

num_vals = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

preprocess = ColumnTransformer(transformers = [
    ('num', num_vals, num_cols)
])

pipeline = Pipeline([
    ('preprocess', preprocess),
    ('rf', RandomForestRegressor())
])

In [175]:
pipeline.fit(x_train, y_train)
print(pipeline.score(x_test, y_test))

0.8060156436162591


In [186]:
search_space = [{'rf': [RandomForestRegressor()],
                 'rf__n_estimators': [100, 200],
                 'rf__max_depth': [10, 100, 1000],
                 'rf__max_features': ['auto', 'sqrt', 'log2'],
                 'rf__min_samples_split': [2, 5, 10],
                 'rf__min_samples_leaf': [1, 2, 4],
                 'rf__bootstrap': [True, False]},
                 {'rf': [LogisticRegression()], 'rf__C': [0.1, 1, 10]},
                 {'rf': [LinearRegression()]}, 
                 {'rf': [Ridge()], 'rf__alpha':
                 [0, 0.01, 0.1, 1, 10, 100]},
                 {'rf': [Lasso()], 'rf__alpha':
                 [0, 0.01, 0.1, 1, 10, 100]},
                 {'rf': [SVR()],
                  'rf__C': [0.1, 1, 10],
                  'rf__gamma': [0.1, 1, 10]},
                 {'rf': [KNeighborsRegressor()],
                  'rf__n_neighbors': [3, 5, 7]},
                 {'rf': [DecisionTreeRegressor()],
                  'rf__max_depth': [10, 100, 1000]},
                 {'rf': [LDA()]}
                 ]


In [163]:
gs = GridSearchCV(pipeline, search_space, cv=5)
gs.fit(x_train, y_train)
best_model = gs.best_estimator_
print(best_model.named_steps['rf'])
print(best_model.named_steps['rf'].get_params())


c:\Users\Asus\anaconda3\Lib\site-packages\sklearn\base.py:1473: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  return fit_method(estimator, *args, **kwargs)
c:\Users\Asus\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
c:\Users\Asus\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.106e+04, tolerance: 5.268e+00 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_d

RandomForestRegressor(max_depth=10, max_features='log2', min_samples_leaf=2)
{'bootstrap': True, 'ccp_alpha': 0.0, 'criterion': 'squared_error', 'max_depth': 10, 'max_features': 'log2', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 2, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'monotonic_cst': None, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': None, 'verbose': 0, 'warm_start': False}


In [166]:
print(best_model.score(x_test, y_test))

0.836111770251449
